In [1]:
from citipy import citipy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from config1 import weatherkey
import requests
import time
from datetime import datetime

In [2]:
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
x = [25.12903645, 25.92017388, 26.62509167, -59.98969384, 37.30571269]
y = [-67.59741259, 11.09532135, 74.84233102, -76.89176677, -61.13376282]
coordinates = zip(x, y)

In [4]:
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

25.12903645 -67.59741259
25.92017388 11.09532135
26.62509167 74.84233102
-59.98969384 -76.89176677
37.30571269 -61.13376282


In [5]:
coordinates = list(lat_lngs)

In [6]:
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    if city not in cities:
        cities.append(city)
len(cities)

595

In [7]:

city = 'Boston'
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weatherkey
city_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weatherkey + "&q=" + city
boston_data = requests.get(city_url).json()
boston_data

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 37.6,
  'feels_like': 33.64,
  'temp_min': 31.77,
  'temp_max': 41.43,
  'pressure': 1020,
  'humidity': 68},
 'visibility': 10000,
 'wind': {'speed': 5.01, 'deg': 292, 'gust': 8.99},
 'clouds': {'all': 90},
 'dt': 1638674427,
 'sys': {'type': 2,
  'id': 2013408,
  'country': 'US',
  'sunrise': 1638619039,
  'sunset': 1638652331},
 'timezone': -18000,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [8]:
lat = boston_data['coord']['lat']
lng = boston_data['coord']['lon']
max_temp = boston_data['main']['temp_max']
humidity = boston_data['main']['humidity']
clouds = boston_data['clouds']['all']
wind = boston_data['wind']['speed']
print(lat,lng,max_temp,humidity,clouds,wind)

42.3584 -71.0598 41.43 68 90 5.01


In [9]:
date = boston_data['dt']
datetime.utcfromtimestamp(date)

datetime.datetime(2021, 12, 5, 3, 20, 27)

In [10]:
datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')

'2021-12-05 03:20:27'

In [11]:
city_data = []
print("Beginning Data Retrieval     \n-----------------------------")
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [12]:
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | minab
Processing Record 2 of Set 1 | grindavik
Processing Record 3 of Set 1 | hermanus
Processing Record 4 of Set 1 | richards bay
Processing Record 5 of Set 1 | shahrud
Processing Record 6 of Set 1 | butaritari
Processing Record 7 of Set 1 | bredasdorp
Processing Record 8 of Set 1 | hithadhoo
Processing Record 9 of Set 1 | hovd
Processing Record 10 of Set 1 | mahanoro
Processing Record 11 of Set 1 | victoria
Processing Record 12 of Set 1 | calbuco
Processing Record 13 of Set 1 | qaanaaq
Processing Record 14 of Set 1 | busselton
Processing Record 15 of Set 1 | wattegama
Processing Record 16 of Set 1 | mahebourg
Processing Record 17 of Set 1 | butterworth
Processing Record 18 of Set 1 | rikitea
Processing Record 19 of Set 1 | airai
Processing Record 20 of Set 1 | rumphi
Processing Record 21 of Set 1 | mataura
Processing Record 22 of Set 1 | tombouctou
Processing Record 23 of Set 1 | dikson
Processing Record 24 of Set 1 | sitka
Processing Record 25 of Set 1

Processing Record 43 of Set 4 | mandurah
Processing Record 44 of Set 4 | fortuna
Processing Record 45 of Set 4 | winburg
Processing Record 46 of Set 4 | saint george
Processing Record 47 of Set 4 | ushumun
Processing Record 48 of Set 4 | ilulissat
Processing Record 49 of Set 4 | dubbo
Processing Record 50 of Set 4 | mombetsu
Processing Record 1 of Set 5 | jutai
Processing Record 2 of Set 5 | yarovoye
Processing Record 3 of Set 5 | lusambo
Processing Record 4 of Set 5 | buala
Processing Record 5 of Set 5 | mumford
Processing Record 6 of Set 5 | saint-louis
Processing Record 7 of Set 5 | amazar
Processing Record 8 of Set 5 | geilo
Processing Record 9 of Set 5 | mehamn
Processing Record 10 of Set 5 | attawapiskat
City not found. Skipping...
Processing Record 11 of Set 5 | borovskoy
Processing Record 12 of Set 5 | porto novo
Processing Record 13 of Set 5 | sakakah
Processing Record 14 of Set 5 | san lawrenz
Processing Record 15 of Set 5 | tshikapa
Processing Record 16 of Set 5 | pacific gr

Processing Record 31 of Set 8 | rodrigues alves
Processing Record 32 of Set 8 | vacaria
Processing Record 33 of Set 8 | kavieng
Processing Record 34 of Set 8 | burica
City not found. Skipping...
Processing Record 35 of Set 8 | churapcha
Processing Record 36 of Set 8 | nakapiripirit
Processing Record 37 of Set 8 | vardo
Processing Record 38 of Set 8 | anchorage
Processing Record 39 of Set 8 | ekhabi
Processing Record 40 of Set 8 | yumen
Processing Record 41 of Set 8 | ciudad del maiz
Processing Record 42 of Set 8 | caravelas
Processing Record 43 of Set 8 | berlevag
Processing Record 44 of Set 8 | paradwip
City not found. Skipping...
Processing Record 45 of Set 8 | hohhot
Processing Record 46 of Set 8 | carroll
Processing Record 47 of Set 8 | buraydah
Processing Record 48 of Set 8 | jiangyou
Processing Record 49 of Set 8 | nizhnevartovsk
Processing Record 50 of Set 8 | kabalo
Processing Record 1 of Set 9 | bestobe
Processing Record 2 of Set 9 | roma
Processing Record 3 of Set 9 | cockbur

Processing Record 19 of Set 12 | acarau
Processing Record 20 of Set 12 | mufulira
Processing Record 21 of Set 12 | sorland
Processing Record 22 of Set 12 | kangaatsiaq
Processing Record 23 of Set 12 | kiruna
Processing Record 24 of Set 12 | ust-nera
Processing Record 25 of Set 12 | los llanos de aridane
Processing Record 26 of Set 12 | ancud
Processing Record 27 of Set 12 | vallenar
Processing Record 28 of Set 12 | tiruvottiyur
City not found. Skipping...
Processing Record 29 of Set 12 | dutse
Processing Record 30 of Set 12 | portoviejo
Processing Record 31 of Set 12 | winnemucca
Processing Record 32 of Set 12 | pogradec
Processing Record 33 of Set 12 | ambulu
Processing Record 34 of Set 12 | suzaka
Processing Record 35 of Set 12 | lodja
Processing Record 36 of Set 12 | mabaruma
Processing Record 37 of Set 12 | gharo
Processing Record 38 of Set 12 | ovsyanka
Processing Record 39 of Set 12 | adrar
Processing Record 40 of Set 12 | miri
Processing Record 41 of Set 12 | lumeje
Processing R

In [13]:
len(city_data)

552